In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/Dashboard/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/Dashboard/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Dashboard/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid//Dashboard/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.ObjectId,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-21T00:00:00Z,01001,"12.05.2021, 00:00 Uhr",0,-9,2021-03-13T00:00:00Z,0,1,1,Nicht übermittelt
1,Feature,None,2,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-21T00:00:00Z,01001,"12.05.2021, 00:00 Uhr",0,-9,2021-03-19T00:00:00Z,0,1,1,Nicht übermittelt
2,Feature,None,3,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-21T00:00:00Z,01001,"12.05.2021, 00:00 Uhr",0,-9,2021-03-21T00:00:00Z,0,1,0,Nicht übermittelt
3,Feature,None,4,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-22T00:00:00Z,01001,"12.05.2021, 00:00 Uhr",0,-9,2021-03-22T00:00:00Z,0,1,0,Nicht übermittelt
4,Feature,None,5,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021-03-23T00:00:00Z,01001,"12.05.2021, 00:00 Uhr",0,-9,2021-03-19T00:00:00Z,0,1,1,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-21,90164.0,...,1,0,0,0,0,1,0,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-21,90164.0,...,1,0,0,0,0,1,0,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-21,90164.0,...,1,0,0,0,0,1,0,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-22,90164.0,...,1,0,0,0,0,1,0,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A15-A34,0,M,2021-03-23,90164.0,...,1,0,0,0,0,1,0,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


#df_bund.set_index('Region', append=True, inplace=True)
#df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)

df_bund.reset_index(inplace=True)
df_bund['Region'] = 'Deutschland'
df_bund['Typ'] = 'Nation'


df_bund.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt,Fall_Last7,Fall_MA7,R7,Einwohner,Fall_100K,Region,Typ
455,2021-05-07,15340,13,7736,7453,151,627,1855,4745,5742,1940,421,10,105249.0,15035.571429,0.86,83166711,126.55,Deutschland,Nation
456,2021-05-08,11938,8,5884,5924,130,609,1510,3571,4467,1467,301,13,102052.0,14578.857143,0.87,83166711,122.71,Deutschland,Nation
457,2021-05-09,5192,5,2645,2488,59,250,596,1675,1890,612,162,7,100547.0,14363.857143,0.89,83166711,120.90,Deutschland,Nation
458,2021-05-10,6697,5,3391,3228,78,240,744,2073,2546,864,215,15,98119.0,14017.000000,0.90,83166711,117.98,Deutschland,Nation
459,2021-05-11,12319,4,6226,5957,136,475,1417,3735,4886,1485,301,20,89474.0,12782.000000,0.85,83166711,107.58,Deutschland,Nation


In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

#df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

#

df_land.reset_index(inplace=True)
df_land['Region'] = df_land['Bundesland']
df_land['Typ'] = 'Bundesland'

df_land.tail()

,Bundesland,Meldedatum,IdBundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
6858,Thüringen,2021-05-07,16,607,2133378,2,272,334,1,17,...,30,0,712.50,581.857143,1.366906,0.86,4073.0,190.92,Thüringen,Bundesland
6859,Thüringen,2021-05-08,16,450,2133378,1,221,227,2,19,...,17,0,624.00,569.857143,1.231376,0.86,3989.0,186.98,Thüringen,Bundesland
6860,Thüringen,2021-05-09,16,214,2133378,0,115,98,1,7,...,12,0,484.25,556.714286,0.854810,0.86,3897.0,182.67,Thüringen,Bundesland
6861,Thüringen,2021-05-10,16,316,2133378,0,169,146,1,11,...,15,1,396.75,547.142857,0.604341,0.88,3830.0,179.53,Thüringen,Bundesland
6862,Thüringen,2021-05-11,16,557,2133378,0,264,291,2,26,...,21,3,384.25,511.857143,0.539298,0.88,3583.0,167.95,Thüringen,Bundesland


In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


#df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.reset_index(inplace=True)
df_kreis['Region'] = df_kreis['Landkreis']
df_kreis['Typ'] = 'Landkreis'

df_kreis.tail()

,Landkreis,Meldedatum,IdLandkreis,Bundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
133777,StadtRegion Aachen,2021-05-07,05334,Nordrhein-Westfalen,114,557026.0,0,53,61,0,...,3,0,127.75,98.285714,1.494152,0.77,688.0,123.51,StadtRegion Aachen,Landkreis
133778,StadtRegion Aachen,2021-05-08,05334,Nordrhein-Westfalen,54,557026.0,1,27,27,0,...,3,0,101.50,94.142857,1.208333,0.78,659.0,118.31,StadtRegion Aachen,Landkreis
133779,StadtRegion Aachen,2021-05-09,05334,Nordrhein-Westfalen,26,557026.0,0,12,14,0,...,2,0,68.75,91.857143,0.670732,0.80,643.0,115.43,StadtRegion Aachen,Landkreis
133780,StadtRegion Aachen,2021-05-10,05334,Nordrhein-Westfalen,49,557026.0,0,33,16,0,...,0,0,60.75,91.428571,0.541203,0.87,640.0,114.90,StadtRegion Aachen,Landkreis
133781,StadtRegion Aachen,2021-05-11,05334,Nordrhein-Westfalen,84,557026.0,0,40,44,0,...,2,0,53.25,80.714286,0.416830,0.82,565.0,101.43,StadtRegion Aachen,Landkreis


In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged['Bundesland'] == 'Berlin', 'IdLandkreis'] = '11000'

df_merged.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,...,Einwohner,Fall_100K,Region,Typ,Bundesland,IdBundesland,Fall_MA4,R4,Landkreis,IdLandkreis
133777,2021-05-07,114,0,53,61,0,6,8,50,36,...,557026.0,123.51,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,127.75,1.494152,StadtRegion Aachen,05334
133778,2021-05-08,54,1,27,27,0,1,3,23,16,...,557026.0,118.31,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,101.50,1.208333,StadtRegion Aachen,05334
133779,2021-05-09,26,0,12,14,0,0,1,8,10,...,557026.0,115.43,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,68.75,0.670732,StadtRegion Aachen,05334
133780,2021-05-10,49,0,33,16,0,0,3,24,19,...,557026.0,114.90,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,60.75,0.541203,StadtRegion Aachen,05334
133781,2021-05-11,84,0,40,44,0,3,11,25,28,...,557026.0,101.43,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,53.25,0.416830,StadtRegion Aachen,05334


In [11]:
if os.path.exists('covid_panel_backup.csv'):
    os.remove('covid_panel_backup.csv')

In [12]:
os.rename('covid_panel.csv', 'covid_panel_backup.csv')

In [13]:
df_merged.to_csv('covid_panel.csv')